# Olist Sales & Revenue Analysis (Project 1)

This notebook regenerates the processed KPI outputs and figures for the portfolio project.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
BASE = Path('.')
RAW = BASE
OUT = BASE/'data_processed'
FIG = BASE/'figures'
OUT.mkdir(exist_ok=True)
FIG.mkdir(exist_ok=True)


## Load CSVs
Place the raw Kaggle files in the project root (same level as this notebook), or update these paths.


In [ ]:
customers = pd.read_csv(RAW/'olist_customers_dataset.csv')
orders = pd.read_csv(RAW/'olist_orders_dataset.csv')
items = pd.read_csv(RAW/'olist_order_items_dataset.csv')
payments = pd.read_csv(RAW/'olist_order_payments_dataset.csv')
products = pd.read_csv(RAW/'olist_products_dataset.csv')
cat_trans = pd.read_csv(RAW/'product_category_name_translation.csv')


In [ ]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'], errors='coerce')
orders_del = orders[orders['order_status']=='delivered'].copy()
items_del = items.merge(orders_del[['order_id','customer_id','order_purchase_timestamp']], on='order_id', how='inner')
items_del['line_revenue'] = items_del['price'].fillna(0) + items_del['freight_value'].fillna(0)
products2 = products.merge(cat_trans, on='product_category_name', how='left')
items_del = items_del.merge(products2[['product_id','product_category_name','product_category_name_english']], on='product_id', how='left')
items_del = items_del.merge(customers[['customer_id','customer_city','customer_state']], on='customer_id', how='left')


In [ ]:
order_kpi = (items_del.groupby('order_id', as_index=False)
             .agg(order_purchase_timestamp=('order_purchase_timestamp','min'),
                  customer_id=('customer_id','first'),
                  items=('order_item_id','count'),
                  revenue=('line_revenue','sum')))
order_kpi['order_month'] = order_kpi['order_purchase_timestamp'].dt.to_period('M').dt.to_timestamp()
monthly = (order_kpi.groupby('order_month', as_index=False)
           .agg(orders=('order_id','nunique'), revenue=('revenue','sum'), aov=('revenue','mean')))
monthly['mom_growth'] = monthly['revenue'].pct_change()
monthly.to_csv(OUT/'monthly_kpis.csv', index=False)


In [ ]:
plt.figure()
plt.plot(monthly['order_month'], monthly['revenue'])
plt.title('Monthly Revenue (Delivered Orders)')
plt.xlabel('Month')
plt.ylabel('Revenue (BRL)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(FIG/'monthly_revenue.png', dpi=200)
plt.close()
